# Model Calibration
## Why calibrate?
Calibration is the process of fine-tuning an agent-based model's input parameters so that its simulated outputs align with real-world data. Without calibration, the model might generate plausible-looking behavior that doesn’t actually reflect reality. Calibration improves the model’s credibility, predictive power, and policy relevance by ensuring it reproduces key observed patterns or outcomes from historical data.

## Calibration Process Overview
1. Define Calibration Targets:
Select measurable real-world data points (e.g., population size, disease prevalence, contraceptive use) that the model should replicate.

2. Identify Parameters to Tune:
Choose uncertain model parameters that strongly influence the outcomes but lack precise empirical estimates (e.g., agent behavior probabilities, environmental factors, etc.).

3. Choose a Calibration Method:
Use either:
- Manual/heuristic tuning (trial-and-error or expert knowledge), or
- Automated optimization (e.g., grid search, random search, genetic algorithms, or Bayesian methods) to systematically explore parameter space.

4. Define a Goodness-of-Fit Metric:
Quantify how well the model output matches the targets using metrics like root mean squared error (RMSE), likelihood scores, or custom error functions.

5. Run the Calibration:
Simulate the model repeatedly with different parameter values and evaluate performance using the fit metric.

6. Select Best-Fit Parameters:
Identify parameter sets that produce model outputs closest to observed data.

7. Validate (if possible):
Use separate data not involved in calibration to assess the model’s generalizability.

In this tutorial, we will walk through the basics of running both a manual and automated calibration. Choosing between manual and automated calibration depends on the model complexity, parameter uncertainty, available data, and computational resources. 

## Manual Calibration
Manual calibration involves adjusting parameters by hand based on expert knowledge or visual inspection of outputs. This can make sense when:
- The model has few parameters to tune
- You have strong domain knowledge about parameter ranges
- You are in the early stages of model development or prototyping
- The simulation is computationally expensive or you have limited computational resources
- You want to explore model behavior qualitatively

The plotting class (in plotting.py) can be used to visually inspect the outputs of common target parameters (e.g. CPR, method mix, TFR, etc.) and compare the model output vs real-world data.

## Automated Calibration
Automated calibration uses optimization algorithms (like Optuna’s Bayesian optimization) to efficiently search parameter space for the best fit. An automated calibration in FPsim uses the calibration and experiment classes to use Optuna's optimization methods to determine the best free parameters. This makes sense to use when:
- Your model has many uncertain parameters (e.g. 5+)
- You have access to compute resources to run many simulations (e.g. in parallel on a machine or VM with ample processing power, memory, and storage OR on a cloud computing platform)
- You have a large number of target parameters to which you want to calibrate with an unbiased approach
 
A hybrid approach can also be to start with a manual calibration to narrow down plausible ranges and understand model dynamics. Then switch to automated methods to fine-tune parameters and formalize the process. 

## Preparing the data

In order to run a calibration successfully, we need to ensure that the fpsim/locations directory contains a directory for the country being calibrated (i.e. 'fpsim/locations/kenya'). This directory should also contain:
- A model file (i.e. fpsim/locations/kenya/kenya.py)
- A data subdirectory with data for the desired calibration targets (see fpsim/locations/README.md for specific files and means of generating each), ideally with the most recently available comprehensive data to compare with the model output 

Ensure that the data in the aforementioned files are formatted in the same manner as those in `locations/kenya/data`.

## Running a Manual Calibration
### Imports
First, we import any needed packages. We also import the plotting class, which is useful in visually inspecting the model vs observed data.

In [ ]:
import fpsim as fp
from fpsim import plotting as plt
import numpy as np

### Setting Parameters

First, we set up our parameters for the simulation(s) used for calibration, including the country name and any specific sim params, such as the population size and start/end year of the sim.

In [ ]:
country = 'kenya'
pars = fp.all_pars(location=country)  # For default pars

Next we set our free parameters to initial values that we will iteratively tune to optimize the model outputs (to be as close as possible to real-world data). The free parameters below are used for tuning:
- fecundity_var_low, fecundity_var_high
- exposure_factor
- spacing_pref
- primary_infertility
- age-based exposure (modified in {country}.py)
- parity-based exposure (modified in {country}.py)

We can also modify the contraceptive choice parameters, which can be useful especially in adjusting the model contraceptive prevalence rate. The `prob_use_year` parameter is helpful to adjust the CPR starting point (seen in the CPR trend plot), and the `prob_use_trend_par` parameter is helpful to adjust the slope in the CPR trend plot. Lastly, the `method_weights` array parameter is useful in tuning the method mix, for example - increasing the % of pill use and decreasing the % of IUD use. 

In [ ]:
# Initial free parameters for calibration
pars['exposure_factor'] = 1

# Postpartum sexual activity correction or 'birth spacing preference'. Pulls values from {location}/data/birth_spacing_pref.csv by default
# Set all to 1 to reset. Option to use 'optimize-space-prefs.py' script in this directory to determine values
pars['spacing_pref']['preference'][:3] =  1  # Spacing of 0-6 months
pars['spacing_pref']['preference'][3:6] = 1  # Spacing of 9-15 months
pars['spacing_pref']['preference'][6:9] = 1  # Spacing of 18-24 months
pars['spacing_pref']['preference'][9:] =  1  # Spacing of 27-36 months

# Only other simulation free parameters are age-based exposure and parity-based exposure (which you can adjust manually in {country}.py) as well as primary_infertility (set to 0.05 by default)

# Adjust contraceptive choice parameters
cm_pars = dict(
    prob_use_year=2020,  # Time trend intercept
    prob_use_trend_par=0.06,   # Time trend parameter
    method_weights=np.array([1, 1, 1, 1, 1, 1, 1, 1, 1])  # Weights for the methods in method_list in methods.py (excluding 'none', so starting with 'pill' and ending in 'othmod').
)
method_choice = fp.SimpleChoice(pars=cm_pars, location=country)     # The contraceptive choice module used (see methods.py for more documentation). We can select RandomChoice, SimpleChoice, or StandardChoice (StandardChoice is selected by default).

### Running the Simulation

We run the simulation with the free parameters specified above:

In [ ]:
# Run the sim
sim = fp.Sim(pars=pars, contraception_module=method_choice)
sim.run()

### Plotting the Target Parameters

Once the sim run completes, we plot the sim results and the target parameters (comparing the model results vs real-world data). 

In [ ]:
# Plot sim
sim.plot()

# Plotting class function which plots the primary calibration targets (method mix, method use, cpr, total fertility rate, birth spacing, age at first birth, and age-specific fertility rate)
plt.plot_calib(sim)

We can see from the plots above that compared to the real-world data in 2020, our model has some adjustments that need to be made. We do this iteratively to see how we can get specific target parameters closer to reality. For example, the model currently results in a TFR that is too low, and the birth space bins have discrepancies (0-12mo is too high, 12-24mo is too high, and 24-48mo is too low). We can modify the free parameters to tune the model to adjust accordingly. Let's try increasing the exposure factor to increase the TFR and modifying the spacing preference factors to account for the current model/data differences:

In [ ]:
# Initial free parameters for calibration
pars['exposure_factor'] = 1

# Last free parameter, postpartum sexual activity correction or 'birth spacing preference'. Pulls values from {location}/data/birth_spacing_pref.csv by default
# Set all to 1 to reset. Option to use 'optimize-space-prefs.py' script in this directory to determine values
pars['spacing_pref']['preference'][:4] = .4  # Spacing of 0-12 months
pars['spacing_pref']['preference'][4:8] = .2  # Spacing of 12-24 months
pars['spacing_pref']['preference'][8:16] = 2  # Spacing of 24-48 months
pars['spacing_pref']['preference'][16:] = 1  # Spacing of >48 months

In [ ]:
# Re-run the sim
method_choice = fp.SimpleChoice(pars=cm_pars, location=country) 
sim = fp.Sim(pars=pars, contraception_module=method_choice)
sim.run()
plt.plot_calib(sim)

Our adjustments helped move the model results towards the real-world trends thankfully! As we iteratively modify the free parameters, we can both improve the calibration of the model and simultaneously learn how the parameters affect the model behavior. For example, as increasing the exposure_factor increased the TFR (which is still too low), we can increase it again in hopes of making it closer to the data. 

There are different strategies of fine-tuning these parameters manually, but one method is to modify the free parameters to get 1-2 target parameters as close as we can before then focusing on a different target parameter to improve via additional free parameter modifications. As these models are quite dynamic, changing one free parameter will often change several target parameters (some more significantly than others); thus, it's worth taking note of which target parameters change and in what direction(s). 

There is an initial learning curve to understanding which free parameters affect which target parameters, but tuning the model by visual inspection of model vs data in the plots becomes easier and quicker over time.

Let's try increasing the exposure_factor once more and also modifying one of the contraceptive choice parameters (`method_weights`) to calibrate the method mix (increasing weights for those with percentages too low and decreasing weights for those with percentages too high):

In [ ]:
# Initial free parameters for calibration
pars['exposure_factor'] = 1

# Last free parameter, postpartum sexual activity correction or 'birth spacing preference'. Pulls values from {location}/data/birth_spacing_pref.csv by default
# Set all to 1 to reset. Option to use 'optimize-space-prefs.py' script in this directory to determine values
pars['spacing_pref']['preference'][:4] = .4  # Spacing of 0-12 months
pars['spacing_pref']['preference'][4:8] = .2  # Spacing of 12-24 months
pars['spacing_pref']['preference'][8:16] = 2  # Spacing of 24-48 months
pars['spacing_pref']['preference'][16:] = 1  # Spacing of >48 months

# Adjust contraceptive choice parameters
cm_pars = dict(
    prob_use_year=2020,  # Time trend intercept
    prob_use_trend_par=0.06,   # Time trend parameter
    force_choose=False,        # Whether to force non-users to choose a method ('False' by default)
    method_weights=np.array([.5, .5, 1, .7, 1, 1, 1.3, .8, 3])  # Weights for the methods in method_list in methods.py (excluding 'none', so starting with 'pill' and ending in 'othmod').
)
method_choice = fp.SimpleChoice(pars=cm_pars, location=country)

In [ ]:
# Re-run the sim
sim = fp.Sim(pars=pars, contraception_module=method_choice)
sim.run()
plt.plot_calib(sim)

Both of these changes helped us improve these target parameters! The TFR (and ASFR) both shifted closer to the data, and the method mix looks much closer as well. As was mentioned however, changing free parameters doesn't necessarily adjust target parameters 1-1 (they rarely do!); notice how these changes resulted in the birth space bin of 12-24mo jumping back up when we had tuned it to be lower (and closer to the data) previously. The good news is that we already learned how to adjust the birth space bins by modifying the spacing_pref param; thus, we can keep the free parameters we've adjusted so far and decrease the spacing_pref 12-24mo weight again to continue the calibration. 

We continue in this manner, iteratively improving the target parameters to which we are calibrating. We can either calibrate solely in this manner (if limited in computational resources and/or in the early stages of model development), or we can perform an approximate calibration and then use the chosen free parameters to help narrow the parameter ranges we want to sweep in an automatic calibration. Automatic calibration will be covered in a forthcoming tutorial.